In [9]:
import pandas as pd


In [44]:
df = pd.read_csv("../data/solar_data_open.csv", parse_dates=["DateTime"])
df

,Date,Time,DateTime,Country,DC_Power,AC_Power,Ambient_Temperature,Module_Temperature,Irradiation,Day,Month,Hour,Daily_Yield,Total_Yield,Price,Demand,Demand_Energy
0,1/1/2024,0,2024-01-01 00:00:00,Australia,0.000,0.000,8.32,14.89,0.015,1,1,0,0.000,0.000,65.840833,6197.7025,6202.22
1,1/1/2024,1,2024-01-01 01:00:00,Australia,0.000,0.000,7.74,11.05,0.000,1,1,1,0.000,0.000,51.865833,6080.4650,6088.46
2,1/1/2024,2,2024-01-01 02:00:00,Australia,0.000,0.000,8.57,12.85,0.016,1,1,2,0.000,0.000,53.718333,5882.9917,5889.73
3,1/1/2024,3,2024-01-01 03:00:00,Australia,0.000,0.000,7.65,14.87,0.004,1,1,3,0.000,0.000,51.587500,5759.6550,5762.59
4,1/1/2024,4,2024-01-01 04:00:00,Australia,0.000,0.000,7.45,12.55,0.000,1,1,4,0.000,0.000,45.140833,5735.6175,5739.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30/12/2024,19,2024-12-30 19:00:00,Australia,9.323,8.425,7.56,13.07,0.155,30,12,19,501.314,151393.705,139.457500,8090.4058,8104.76
8756,30/12/2024,20,2024-12-30 20:00:00,Australia,0.000,0.000,8.89,14.77,0.000,30,12,20,501.314,151393.705,121.981670,7739.1792,7752.98
8757,30/12/2024,21,2024-12-30 21:00:00,Australia,0.000,0.000,8.76,12.69,0.000,30,12,21,501.314,151393.705,106.763330,7526.3025,7536.24
8758,30/12/2024,22,2024-12-30 22:00:00,Australia,0.000,0.000,7.69,13.26,0.008,30,12,22,501.314,151393.705,106.674170,6992.3942,7016.85


In [22]:
print(df.describe())
print(df.info())

                  Datetime      AC_Power        Price  Battery_State_kWh  \
count                 8784  8.784000e+03  8784.000000        8784.000000   
mean   2024-07-01 23:30:00  9.932450e+00     0.150953           9.861982   
min    2024-01-01 00:00:00 -8.881784e-16     0.050000           0.000000   
25%    2024-04-01 11:45:00  6.429317e+00     0.117074          10.000000   
50%    2024-07-01 23:30:00  9.879188e+00     0.150754          10.000000   
75%    2024-10-01 11:15:00  1.333196e+01     0.183777          10.000000   
max    2024-12-31 23:00:00  2.963119e+01     0.373954          10.000000   
std                    NaN  4.984412e+00     0.048637           0.821746   

        Charge_kwh  Discharge_kwh    Revenue_$  Cumulative_Revenue_$  
count  8784.000000    8784.000000  8784.000000           8784.000000  
mean      0.107013       0.105874     0.027043            110.628501  
min       0.000000       0.000000     0.000000              0.000000  
25%       0.000000       0.0000

In [ ]:
daily_price_stats = df.groupby("Date")["Price"].agg(
    low_threshold=lambda x: x.quantile(0.25),
    high_threshold=lambda x: x.quantile(0.75)
).reset_index()
df = df.merge(daily_price_stats, on="Date", how="left")
# 电池参数配置
battery_config = {
    'battery_capacity_kwh': 13.3,
    'battery_price_total': 8500,
    'round_trip_efficiency': 0.90,
    'max_charge_discharge_rate_kwh': 7,
    'annual_maintenance_cost': 170,
    'lifespan_years': 10,
}

# 提取参数
capacity = battery_config['battery_capacity_kwh']
efficiency = battery_config['round_trip_efficiency']
max_rate = battery_config['max_charge_discharge_rate_kwh']
# 初始化电池状态和结果容器
battery_state = 0.0
results = []

# 模拟每个小时的充放电
for i in range(len(df)):
    timestamp = df.loc[i, "DateTime"]
    power = df.loc[i, "AC_Power"]
    price = df.loc[i, "Price"]
    low_th = df.loc[i, "low_threshold"]
    high_th = df.loc[i, "high_threshold"]
    charge_kwh = 0.0
    discharge_kwh = 0.0
    revenue = 0.0
    
    # 1. 白天有发电
    if power > 0:
        # 如果电价低（便宜 or 负价），先充电
        if price <= low_th:
            charge_possible = min(power, max_rate, capacity - battery_state)
            battery_state += charge_possible * efficiency
            charge_kwh = charge_possible
            sell_to_grid = power - charge_possible
        else:
            # 直接全卖
            sell_to_grid = power
        revenue = sell_to_grid * price / 1000

    # 2. 夜间无发电
    else:
        if price >= high_th and battery_state > 0:
            discharge_possible = min(max_rate, battery_state)
            battery_state -= discharge_possible
            discharge_kwh = discharge_possible
            revenue = discharge_possible * price / 1000

    # 存储模拟结果
    results.append({
        "Datetime": timestamp,
        "AC_Power": power,
        "Price": price,
        "Battery_State_kWh": battery_state,
        "Charge_kwh": charge_kwh,
        "Discharge_kwh": discharge_kwh,
        "Revenue_$": revenue
    })

# 创建新 DataFrame
simulated_df = pd.DataFrame(results)
simulated_df["Cumulative_Revenue_$"] = simulated_df["Revenue_$"].cumsum()

# 查看结果
simulated_df

,Datetime,AC_Power,Price,Battery_State_kWh,Charge_kwh,Discharge_kwh,Revenue_$,Cumulative_Revenue_$
0,2024-01-01 00:00:00,0.000,65.840833,0.000000,0.0,0.0,0.000000,0.000000
1,2024-01-01 01:00:00,0.000,51.865833,0.000000,0.0,0.0,0.000000,0.000000
2,2024-01-01 02:00:00,0.000,53.718333,0.000000,0.0,0.0,0.000000,0.000000
3,2024-01-01 03:00:00,0.000,51.587500,0.000000,0.0,0.0,0.000000,0.000000
4,2024-01-01 04:00:00,0.000,45.140833,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
8755,2024-12-30 19:00:00,8.425,139.457500,13.299995,0.0,0.0,1.174929,8635.123202
8756,2024-12-30 20:00:00,0.000,121.981670,13.299995,0.0,0.0,0.000000,8635.123202
8757,2024-12-30 21:00:00,0.000,106.763330,13.299995,0.0,0.0,0.000000,8635.123202
8758,2024-12-30 22:00:00,0.000,106.674170,13.299995,0.0,0.0,0.000000,8635.123202


In [57]:
# 提取电池配置参数
capacity = battery_config['battery_capacity_kwh']
efficiency = battery_config['round_trip_efficiency']
max_rate = battery_config['max_charge_discharge_rate_kwh']

# 初始化电池状态和结果容器
battery_state = 0.0
results = []

# 遍历每个小时的数据
for i in range(len(df)):
    timestamp = df.loc[i, "DateTime"]
    power = df.loc[i, "AC_Power"]
    price = df.loc[i, "Price"]
    hour = timestamp.hour
    
    charge_kwh = 0.0
    discharge_kwh = 0.0
    revenue = 0.0

    # 有发电时
    if power > 0:
        # 早上 8~10 点：充电优先
        if 10 <= hour < 12:
            charge_possible = min(power, max_rate, capacity - battery_state)
            battery_state += charge_possible * efficiency
            charge_kwh = charge_possible
            sell_to_grid = power - charge_possible
        else:
            # 其他时间直接卖电
            sell_to_grid = power
        revenue = sell_to_grid * price / 1000

    # 无发电时
    else:
        # 晚上 18~20 点：放电
        if 18 <= hour < 20 and battery_state > 0:
            discharge_possible = min(max_rate, battery_state)
            battery_state -= discharge_possible
            discharge_kwh = discharge_possible
            revenue = discharge_kwh * price / 1000

    # 记录模拟结果
    results.append({
        "Datetime": timestamp,
        "AC_Power": power,
        "Price": price,
        "Battery_State_kWh": battery_state,
        "Charge_kwh": charge_kwh,
        "Discharge_kwh": discharge_kwh,
        "Revenue_$": revenue
    })

# 构建新 DataFrame
simulated_df = pd.DataFrame(results)
simulated_df["Cumulative_Revenue_$"] = simulated_df["Revenue_$"].cumsum()

In [58]:
# 从电池配置中提取参数
battery_cost = battery_config["battery_price_total"]
maintenance_per_year = battery_config["annual_maintenance_cost"]
lifespan = battery_config["lifespan_years"]

# 总投资成本
total_cost = battery_cost + maintenance_per_year * lifespan

# 模拟总收益
total_revenue = simulated_df["Revenue_$"].sum()
total_revenue_10y = total_revenue * lifespan

# ROI 计算
roi = ((total_revenue_10y - total_cost) / total_cost) * 100


print(f"1 year Revenue: ${total_revenue:.2f}")
print(f"10 year Revenue: ${total_revenue_10y:.2f}")
print(f"Total Cost: ${total_cost:.2f}")
print(f"ROI: {roi:.2f}%")

1 year Revenue: $8484.80
10 year Revenue: $84848.01
Total Cost: $10200.00
ROI: 731.84%


In [59]:
simulated_df.to_csv("../data/solar_battery_simulation_results.csv", index=False)

In [ ]:
# Energy Storage System (ESS) Parameters
battery_config = {
    'battery_capacity_kwh': 13.3,          # kWh
    'battery_price_total': 8500,           # Total upfront cost including installation (AUD)
    'round_trip_efficiency': 0.90,         # Charging and discharging efficiency of the system
    'max_charge_discharge_rate_kwh': 5,    # max charge/discharge rate per hour
    'annual_maintenance_cost': 170,        # battery_price_total * 0.02
    'lifespan_years': 10,                  # Expected lifespan of the battery system
}

In [ ]:
# 添加日期列
simulated_df["Date"] = simulated_df["Datetime"].dt.date

# 每天的累计收益
daily_revenue = simulated_df.groupby("Date")["Revenue_$"].sum().cumsum().reset_index()
daily_revenue.columns = ["Date", "Cumulative_Revenue"]

# 将总投资均匀分摊到 10 年的每一天
days_total = lifespan * 365
daily_revenue["Cumulative_Cost"] = total_cost * (daily_revenue.index + 1) / days_total

# ROI 按天
daily_revenue["ROI_%"] = ((daily_revenue["Cumulative_Revenue"] - daily_revenue["Cumulative_Cost"]) / daily_revenue["Cumulative_Cost"]) * 100

daily_revenue.head()

In [60]:
years = 10

simulated_10y = pd.concat([
    simulated_df.assign(
        Datetime=simulated_df["Datetime"] + pd.DateOffset(years=i),
        Year=simulated_df["Datetime"].dt.year + i
    )
    for i in range(years)
], ignore_index=True)

# 重新设置累计收益
simulated_10y["Cumulative_Revenue_$"] = simulated_10y["Revenue_$"].cumsum()

In [61]:
battery_cost = battery_config["battery_price_total"]
maintenance_per_year = battery_config["annual_maintenance_cost"]
lifespan = battery_config["lifespan_years"]
total_cost = battery_cost + maintenance_per_year * lifespan

# 每年总收益
annual_revenue = simulated_10y.groupby("Year")["Revenue_$"].sum().reset_index()
annual_revenue["Cumulative_Revenue"] = annual_revenue["Revenue_$"].cumsum()

# 计算 ROI 每年
annual_revenue["ROI_%"] = (
    (annual_revenue["Cumulative_Revenue"] - total_cost) / total_cost
) * 100

In [63]:
annual_revenue.to_csv("../data/annual_revenue_10_years.csv", index=False)